In [1]:
import os
import zipfile
from os.path import exists

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

from mapie.classification import MapieClassifier
from mapie.metrics import classification_coverage_score
from mapie.metrics import classification_mean_width_score

In [2]:
# import dataset
bean_data_file = "../data/DryBeanDataset/dry_bean_dataset.zip"
if exists(bean_data_file):
    with zipfile.ZipFile(bean_data_file, 'r') as zip_ref:
            zip_ref.extractall(path="../data")

In [3]:
bean_data_file = "../data/DryBeanDataset/Dry_Bean_Dataset.xlsx"
beans = pd.read_excel(bean_data_file, engine='openpyxl')

In [4]:
beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [5]:
le = LabelEncoder()
beans["Class"] = le.fit_transform(beans["Class"])

In [6]:
Y = beans["Class"]
X = beans.drop("Class", axis=1)
X_train, X_rest1, y_train, y_rest1 = train_test_split(X, Y, train_size=10000, random_state=2)
X_test, X_rest2, y_test, y_rest2 = train_test_split(X_rest1, y_rest1, train_size=1000, random_state=42)
X_calib, X_new, y_calib, y_new = train_test_split(X_rest2, y_rest2, train_size=1000, random_state=42)

## 1. Non Conformal prediction

In [33]:
model = GaussianNB().fit(X_train, y_train)

In [34]:
y_pred = model.predict(X_test)
print("Accuracy:", (y_pred == y_test).mean() )

Accuracy: 0.758


In [35]:
X_test.iloc[0:5]

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
7448,34189,687.252,249.062125,174.993660,1.423264,0.711575,34559,208.640329,0.730066,0.989294,0.909629,0.837704,0.007285,0.002213,0.701748,0.998772
9407,47783,820.597,311.415698,196.680458,1.583359,0.775320,48265,246.656046,0.826524,0.990013,0.891710,0.792048,0.006517,0.001582,0.627339,0.993303
9908,51935,864.959,326.225089,203.108862,1.606159,0.782538,52490,257.149170,0.795573,0.989427,0.872326,0.788257,0.006281,0.001496,0.621349,0.997985
5834,47091,874.362,353.032918,171.269691,2.061269,0.874438,47849,244.863479,0.706807,0.984158,0.774043,0.693600,0.007497,0.001070,0.481080,0.991636
5769,45816,859.129,338.735770,173.166110,1.956132,0.859454,46674,241.525864,0.793103,0.981617,0.780028,0.713021,0.007393,0.001179,0.508400,0.994497


In [37]:
le.inverse_transform(model.predict(X_test.iloc[0:5]))

array(['DERMASON', 'SIRA', 'HOROZ', 'HOROZ', 'SIRA'], dtype=object)

In [27]:
cm = confusion_matrix(y_test, y_pred)
print(pd.DataFrame(cm, index=le.classes_, columns=le.classes_))

    0   1   2    3    4    5    6
0  46   0  47    0    6    0    4
1   0  33   0    0    0    0    0
2  20   0  81    0    3    0    0
3   0   0   0  223    0   32    9
4   0   0   4    3  104    0   22
5   2   0   0   26    1  127   22
6   0   0   0   10   10   21  144


## 2. Score mode

In [31]:
alpha = 0.05

def class_wise_performance(y_new, y_set, classes):
    df = pd.DataFrame()
    for i in range(len(classes)):
        ynew = y_new.values[y_new.values == i]
        yscore = y_set[y_new.values == i]
        cov = classification_coverage_score(ynew, yscore)
        size = classification_mean_width_score(yscore)
        
        temp_df = pd.DataFrame({
            "class" : [classes[i]],
            "coverage" : [cov],
            "avg. set size" : [size]
        }, index = [i])
        
        df = pd.concat([df, temp_df])
    return(df)

In [40]:
model = GaussianNB().fit(X_train, y_train)

# Initialize Classifier
mapie_score = MapieClassifier(model, cv="prefit", method="score")

# Calibration
mapie_score.fit(X_calib, y_calib)

# Prediction step
y_pred, y_set = mapie_score.predict(X_new, alpha=alpha)
# remove alpha-dimension
y_set = np.squeeze(y_set)

array([3, 3, 3, ..., 3, 4, 4])

In [33]:
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print("Coverage: {:.2%}".format(cov))
print("Avg. set size: {:2f}\n".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

Coverage: 96.28%
Avg. set size: 1.831782

      class  coverage  avg. set size
0  BARBUNYA  0.925287       2.137931
1    BOMBAY  1.000000       1.000000
2      CALI  0.965909       2.130682
3  DERMASON  0.972906       1.463054
4     HOROZ  0.938525       1.872951
5     SEKER  0.970213       2.017021
6      SIRA  0.974277       1.974277


### 2.1 Conformal by Hand

In [8]:
model = GaussianNB().fit(X_train, y_train)
alpha = 0.05
n = len(X_calib)

In [25]:
# predict heuristic notion of uncertainty from model for calibration 
raw_score_cal = model.predict_proba(X_calib)
raw_score_new = model.predict_proba(X_new)

In [19]:
# 1: get conformal scores. n = calib_Y.shape[0]
cal_scores = 1-raw_score_cal[np.arange(n),y_calib]

In [41]:
# 2: get adjusted quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, interpolation='higher')
prediction_sets = raw_score_new >= (1-qhat) # 3: form prediction sets
prediction = model.predict(X_new)

In [42]:
cov = classification_coverage_score(y_new, prediction_sets)
setsize = classification_mean_width_score(prediction_sets)
print("Coverage: {:.2%}".format(cov))
print("Avg. set size: {:2f}\n".format(setsize))
print(class_wise_performance(y_new, prediction_sets, le.classes_))

Coverage: 96.34%
Avg. set size: 1.837368

      class  coverage  avg. set size
0  BARBUNYA  0.925287       2.149425
1    BOMBAY  1.000000       1.000000
2      CALI  0.965909       2.136364
3  DERMASON  0.972906       1.465517
4     HOROZ  0.938525       1.881148
5     SEKER  0.974468       2.021277
6      SIRA  0.974277       1.980707


## 3. Adaptive prediction set

In [85]:
# Initialize Classifier
mapie_score = MapieClassifier(model, cv="prefit", method="cumulated_score", random_state=1)

# Calibration
mapie_score.fit(X_calib, y_calib)

# Prediction step
y_pred, y_set = mapie_score.predict(X_new, alpha=alpha, include_last_label="randomized")
# remove alpha-dimension
y_set = np.squeeze(y_set)

In [86]:
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print("Coverage: {:.2%}".format(cov))
print("Avg. set size: {:2f}\n".format(setsize))
print(class_wise_performance(y_new, y_set, le.classes_))

Coverage: 95.97%
Avg. set size: 1.890751

      class  coverage  avg. set size
0  BARBUNYA  0.931034       2.212644
1    BOMBAY  1.000000       2.000000
2      CALI  0.960227       2.164773
3  DERMASON  0.972906       1.443350
4     HOROZ  0.934426       1.979508
5     SEKER  0.961702       1.978723
6      SIRA  0.967846       1.980707
